## XBRL US API - ACFR statements by report  

### Authenticate for access token
Click in the gray code cell below, then click the Run button above to execute the cell. Type your XBRL US Web account email, account password, Client ID, and secret as noted, pressing the Enter key on the keyboard after each entry.

XBRL US limits records returned for a query to improve efficiency; this script loops to collect all data from the Public Filings Database for a query. **Non-members might not be able to return all data for a query** - join XBRL US for comprehensive access - https://xbrl.us/join.

In [63]:
print('Enter your XBRL US Web account email: ')
import os, re, sys, json
import requests
import pandas as pd
from IPython.display import display, HTML
import numpy as np
import getpass
from datetime import datetime
import urllib
from urllib.parse import urlencode
email = input()
password = getpass.getpass(prompt='Password: ')
clientid = getpass.getpass(prompt='Client ID: ')
secret = getpass.getpass(prompt='Secret: ')

body_auth = {'username' : ''.join(email),
            'client_id': ''.join(clientid),
            'client_secret' : ''.join(secret),
            'password' : ''.join(password),
            'grant_type' : 'password',
            'platform' : 'ipynb' }

payload = urlencode(body_auth)
url = 'https://api.xbrl.us/oauth2/token'
headers = {"Content-Type": "application/x-www-form-urlencoded"}

res = requests.request("POST", url, data=payload, headers=headers)
auth_json = res.json()

if 'error' in auth_json:
    print ("\n\nThere was a problem generating an access token with these credentials. Run the first cell again to enter credentials.")
else:
    print ("\n\nYour access token expires in 60 minutes. After it expires, run the cell immediately below this one to generate a new token and continue to use the query cell. \n\nFor now, skip ahead to the section 'Make a Query'.")
access_token = auth_json['access_token']
refresh_token = auth_json['refresh_token']
newaccess = ''
newrefresh = ''
#print('access token: ' + access_token + ' refresh token: ' + refresh_token)

Enter your XBRL US Web account email: 


Your access token expires in 60 minutes. After it expires, run the cell immediately below this one to generate a new token and continue to use the query cell. 

For now, skip ahead to the section 'Make a Query'.


#### Refresh token
The cell below is only needed to refresh an expired access token after 60 minutes. When the access token no longer returns results, run the cell below to refresh the access token or re-enter credentials by running the cell above. Until the refresh token process is needed, **skip ahead to _Make a Query_**.


In [83]:
token = token if newrefresh != '' else refresh_token

refresh_auth = {'client_id': ''.join(clientid),
            'client_secret' : ''.join(secret),
            'grant_type' : 'refresh_token',
            'platform' : 'ipynb',
            'refresh_token' : ''.join(token) }
refreshres = requests.post(url, data=refresh_auth)
refresh_json = refreshres.json()
access_token = refresh_json['access_token']
refresh_token = refresh_json['refresh_token']#print('access token: ' + access_token + 'refresh token: ' + refresh_token)
print('Your access token is refreshed for 60 minutes. If it expires again, run this cell to generate a new token and continue to use the query cells below.')
print(access_token)

Your access token is refreshed for 60 minutes. If it expires again, run this cell to generate a new token and continue to use the query cells below.
ac4fb16b-b69d-4c2a-837d-dfd10cbee4ec


### Make a query
After the access token confirmation appears above, you can modify the query below and use the **_Cell >> Run_** menu option with the cell **immediately below this text** to run the query for updated results.

The sample results are from a set of ACFR reports posted to the XBRL US Public Filings Database.  To test for results quickly, modify the **_report\_ids_** to shorten the list, and change the **_XBRL\_Elements_** to return different data from an ACFR statement.
  
Refer to XBRL API documentation at https://xbrlus.github.io/xbrl-api/#/Facts/getFactDetails for other endpoints and parameters to filter and return.

In [84]:
# Define the parameters for the filter and fields to be returned,
# run the loop to return results
offset_value = 0
res_df = []

# Define which endpoint to use
endpoint = 'cube' #taxonomy presentation linkbase + facts

# Define the parameters of the query

# query for ACFR reports, sort by year descending and name ascending
# https://api.xbrl.us/api/v1/report/search?report.source-name=ACFR&fields=report.entity-name,report.period-focus,report.year-focus,report.filing-date,report.id,report.entry-url,report.source-name

report_ids = [# ACFR report ids
'591764'# County of Ogemaw: https://xbrlus.github.io/acfr/ixviewer/ix.html?doc=../samples/100/Ogemaw-20210930-Annual-Accounts.htm
#__pending__ Flint, Michigan: https://xbrlus.github.io/acfr/ixviewer/ix.html?doc=../samples/107/FLINTF652021.htm
#'591765',# William Rainey Harper College: https://xbrlus.github.io/acfr/ixviewer/ix.html?doc=../samples/106/HARPER2021.htm
#'591766',# Oakton Community College: https://xbrlus.github.io/acfr/ixviewer/ix.html?doc=../samples/77/OAKTON2021.html
#'591767' # College of DuPage: https://xbrlus.github.io/acfr/ixviewer/ix.html?doc=../samples/82/COD2021.htm
]

# query for unique Statements in the 2022 GRIP Taxonomy
# https://api.xbrl.us/api/v1/dts/729592/network/search?network.link-name=presentationLink&fields=network.role-description.sort(ASC),dts.id&unique

XBRL_Elements = [
		# remove the Statement between the quotes to get all data for report.ids below
		]

# Define data fields to return (multi-sort based on order)

fields = [ # this is the list of the characteristics of the data being returned by the query
		'report.id',
		'cube.description.sort(ASC)',
		'cube.tree-sequence.sort(ASC)',
		'report.entity-name',
		'dimensions.count',
		'dimension-pair',
		'cube.primary-local-name',
		'fact.value',
		'unit'
        ]

params = { # this is the list of what's being queried against the endpoint
         'cube.description': ','.join(XBRL_Elements),
         'report.id': ','.join(report_ids),
         'fields': ','.join(fields),
         'unique': ''
         }

# Create query and loop for all results - code below does not need to be changed
search_endpoint = 'https://api.xbrl.us/api/v1/' + endpoint + '/search'
orig_fields = params['fields']

count = 0
query_start = datetime.now()
printed = False
while True:
    if not printed:
        printed = True
    res = requests.get(search_endpoint, params=params, headers={'Authorization' : 'Bearer {}'.format(access_token)})
    res_json = res.json()
    if 'error' in res_json:
        print('There was an error: {}'.format(res_json['error_description']))
        break

    print("up to", str(offset_value + res_json['paging']['limit']), "records are found so far ...")

    res_df += res_json['data']

    if res_json['paging']['count'] < res_json['paging']['limit']:
        print(" - this set contained fewer than the", res_json['paging']['limit'], "possible, only", str(res_json['paging']['count']), "records.")
        break
    else:
        offset_value += res_json['paging']['limit']
        if 100 == res_json['paging']['limit']:
                params['fields'] = orig_fields + ',' + endpoint + '.offset({})'.format(offset_value)
                if offset_value == 10 * res_json['paging']['limit']:
                        break
        elif 500 == res_json['paging']['limit']:
                params['fields'] = orig_fields + ',' + endpoint + '.offset({})'.format(offset_value)
                if offset_value == 4 * res_json['paging']['limit']:
                        break
        params['fields'] = orig_fields + ',' + endpoint + '.offset({})'.format(offset_value)

if not 'error' in res_json:
    current_datetime = datetime.now().replace(microsecond=0)
    time_taken = current_datetime - query_start
    index = pd.DataFrame(res_df).index
    total_rows = len(index)
    your_limit = res_json['paging']['limit']
    limit_message = "If the results below match the limit noted above, you might not be seeing all rows, and should consider upgrading (https://xbrl.us/access-token).\n"

    if your_limit == 100:
        print("\nThis non-Member account has a limit of " , 10 * your_limit, " rows per query from our Public Filings Database. " + limit_message)
    elif your_limit == 500:
        print("\nThis Basic Individual Member account has a limit of ", 4 * your_limit, " rows per query from our Public Filings Database. " + limit_message)

    print("\nAt " + current_datetime.strftime("%c") +  ", the query finished with  ", str(total_rows), "  rows returned in " + str(time_taken) + " for \n" +  urllib.parse.unquote(res.url))


    df = pd.DataFrame(res_df)

There was an error: The value entered for report.id of  is not a valid integer. 


In [ ]:
# If you run this program locally, you can save the output to a file on your computer (modify D:\results.csv to your system)
# df.to_csv(r"D:\results.csv",sep=",")

In [79]:
df

,report.id,cube.description,cube.tree-sequence,report.entity-name,dimensions.count,dimension-pair,cube.primary-local-name,fact.value,unit
0,591764,100000 - Statement - Net Position,316,County of Ogemaw,1,[{'TypeOfGovernmentUnitAxis': 'BusinessTypeAct...,AccumulatedDepreciation,-1265638,USD
1,591764,100000 - Statement - Net Position,316,County of Ogemaw,1,[{'TypeOfGovernmentUnitAxis': 'BusinessTypeAct...,AccumulatedDepreciation,-1335962,USD
2,591764,100000 - Statement - Net Position,316,County of Ogemaw,1,[{'TypeOfGovernmentUnitAxis': 'GovernmentalAct...,AccumulatedDepreciation,-6842301,USD
3,591764,100000 - Statement - Net Position,316,County of Ogemaw,1,[{'TypeOfGovernmentUnitAxis': 'GovernmentalAct...,AccumulatedDepreciation,-6895158,USD
4,591764,100000 - Statement - Net Position,338,County of Ogemaw,1,[{'TypeOfGovernmentUnitAxis': 'GovernmentalAct...,CapitalAssetsNotBeingDepreciated,262693,USD
...,...,...,...,...,...,...,...,...,...
1714,591764,805050 - Annual Form F65 MI - Component Unit -...,195,County of Ogemaw,1,[{'TypeOfGovernmentUnitAxis': 'PrimaryGovernme...,NetPositionAtBeginningOfPeriodAfterAdjustments,4585421,USD
1715,591764,805050 - Annual Form F65 MI - Component Unit -...,196,County of Ogemaw,1,[{'TypeOfGovernmentUnitAxis': 'BusinessTypeAct...,NetPosition,8157709,USD
1716,591764,805050 - Annual Form F65 MI - Component Unit -...,196,County of Ogemaw,1,[{'TypeOfGovernmentUnitAxis': 'ComponentUnitDi...,NetPosition,36894568,USD
1717,591764,805050 - Annual Form F65 MI - Component Unit -...,196,County of Ogemaw,1,[{'TypeOfGovernmentUnitAxis': 'GovernmentalAct...,NetPosition,-2925721,USD


In [80]:
# create separate column for cube id
df['cube_id'] = df['cube.description'].apply(lambda x: x.split('-')[0].strip())
df.head()

,report.id,cube.description,cube.tree-sequence,report.entity-name,dimensions.count,dimension-pair,cube.primary-local-name,fact.value,unit,cube_id
0,591764,100000 - Statement - Net Position,316,County of Ogemaw,1,[{'TypeOfGovernmentUnitAxis': 'BusinessTypeAct...,AccumulatedDepreciation,-1265638,USD,100000
1,591764,100000 - Statement - Net Position,316,County of Ogemaw,1,[{'TypeOfGovernmentUnitAxis': 'BusinessTypeAct...,AccumulatedDepreciation,-1335962,USD,100000
2,591764,100000 - Statement - Net Position,316,County of Ogemaw,1,[{'TypeOfGovernmentUnitAxis': 'GovernmentalAct...,AccumulatedDepreciation,-6842301,USD,100000
3,591764,100000 - Statement - Net Position,316,County of Ogemaw,1,[{'TypeOfGovernmentUnitAxis': 'GovernmentalAct...,AccumulatedDepreciation,-6895158,USD,100000
4,591764,100000 - Statement - Net Position,338,County of Ogemaw,1,[{'TypeOfGovernmentUnitAxis': 'GovernmentalAct...,CapitalAssetsNotBeingDepreciated,262693,USD,100000


In [81]:
# df for the balance sheet
balance_df = df[df['cube_id'] == '300690']
balance_df.head()

,report.id,cube.description,cube.tree-sequence,report.entity-name,dimensions.count,dimension-pair,cube.primary-local-name,fact.value,unit,cube_id
455,591764,300690 - Statement - Governmental Funds Balanc...,17,County of Ogemaw,1,[{'GovernmentalFundsAxis': 'AggregateNonmajorF...,CashAndCashEquivalentsModifiedAccrual,1976635,USD,300690
456,591764,300690 - Statement - Governmental Funds Balanc...,17,County of Ogemaw,1,[{'GovernmentalFundsAxis': 'GeneralFundMember'}],CashAndCashEquivalentsModifiedAccrual,37423,USD,300690
457,591764,300690 - Statement - Governmental Funds Balanc...,17,County of Ogemaw,1,[{'GovernmentalFundsAxis': 'GovernmentalFundsM...,CashAndCashEquivalentsModifiedAccrual,3611549,USD,300690
458,591764,300690 - Statement - Governmental Funds Balanc...,39,County of Ogemaw,1,[{'GovernmentalFundsAxis': 'AggregateNonmajorF...,AccountsReceivableNetOfAllowanceModifiedAccrual,22375,USD,300690
459,591764,300690 - Statement - Governmental Funds Balanc...,39,County of Ogemaw,1,[{'GovernmentalFundsAxis': 'GeneralFundMember'}],AccountsReceivableNetOfAllowanceModifiedAccrual,98433,USD,300690


In [56]:
# get the fund member from the dimension pair (general, governmental, etc)
balance_df['dict-pair'] = balance_df['dimension-pair'].apply(lambda x: x[0])
balance_df['fund-member'] = balance_df['dict-pair'].apply(lambda x: x['GovernmentalFundsAxis'])
balance_df.drop(columns=['dict-pair'], inplace=True)
balance_df.head()

/var/folders/w3/7q3wzvhj48n9gsxkkfks4zq00000gn/T/ipykernel_5467/3434965683.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_df['dict-pair'] = balance_df['dimension-pair'].apply(lambda x: x[0])
/var/folders/w3/7q3wzvhj48n9gsxkkfks4zq00000gn/T/ipykernel_5467/3434965683.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  balance_df['fund-member'] = balance_df['dict-pair'].apply(lambda x: x['GovernmentalFundsAxis'])
/var/folders/w3/7q3wzvhj48n9gsxkkfks4zq00000gn/T/ipykernel_5467/3434965683.py:3: Sett

,report.id,cube.description,cube.tree-sequence,report.entity-name,dimensions.count,dimension-pair,cube.primary-local-name,fact.value,unit,cube_id,fund-member
455,591764,300690 - Statement - Governmental Funds Balanc...,17,County of Ogemaw,1,[{'GovernmentalFundsAxis': 'AggregateNonmajorF...,CashAndCashEquivalentsModifiedAccrual,1976635,USD,300690,AggregateNonmajorFundMember
456,591764,300690 - Statement - Governmental Funds Balanc...,17,County of Ogemaw,1,[{'GovernmentalFundsAxis': 'GeneralFundMember'}],CashAndCashEquivalentsModifiedAccrual,37423,USD,300690,GeneralFundMember
457,591764,300690 - Statement - Governmental Funds Balanc...,17,County of Ogemaw,1,[{'GovernmentalFundsAxis': 'GovernmentalFundsM...,CashAndCashEquivalentsModifiedAccrual,3611549,USD,300690,GovernmentalFundsMember
458,591764,300690 - Statement - Governmental Funds Balanc...,39,County of Ogemaw,1,[{'GovernmentalFundsAxis': 'AggregateNonmajorF...,AccountsReceivableNetOfAllowanceModifiedAccrual,22375,USD,300690,AggregateNonmajorFundMember
459,591764,300690 - Statement - Governmental Funds Balanc...,39,County of Ogemaw,1,[{'GovernmentalFundsAxis': 'GeneralFundMember'}],AccountsReceivableNetOfAllowanceModifiedAccrual,98433,USD,300690,GeneralFundMember


In [57]:
# filter for only general fund
general_balance_df = balance_df[balance_df['fund-member'] == 'GeneralFundMember']
general_balance_df.head()

,report.id,cube.description,cube.tree-sequence,report.entity-name,dimensions.count,dimension-pair,cube.primary-local-name,fact.value,unit,cube_id,fund-member
456,591764,300690 - Statement - Governmental Funds Balanc...,17,County of Ogemaw,1,[{'GovernmentalFundsAxis': 'GeneralFundMember'}],CashAndCashEquivalentsModifiedAccrual,37423,USD,300690,GeneralFundMember
459,591764,300690 - Statement - Governmental Funds Balanc...,39,County of Ogemaw,1,[{'GovernmentalFundsAxis': 'GeneralFundMember'}],AccountsReceivableNetOfAllowanceModifiedAccrual,98433,USD,300690,GeneralFundMember
462,591764,300690 - Statement - Governmental Funds Balanc...,58,County of Ogemaw,1,[{'GovernmentalFundsAxis': 'GeneralFundMember'}],PropertyTaxesReceivableModifiedAccrual,895333,USD,300690,GeneralFundMember
465,591764,300690 - Statement - Governmental Funds Balanc...,158,County of Ogemaw,1,[{'GovernmentalFundsAxis': 'GeneralFundMember'}],DueFromStateGovernmentModifiedAccrual,194541,USD,300690,GeneralFundMember
468,591764,300690 - Statement - Governmental Funds Balanc...,161,County of Ogemaw,1,[{'GovernmentalFundsAxis': 'GeneralFundMember'}],DueFromOtherGovernmentModifiedAccrual,59730,USD,300690,GeneralFundMember


In [58]:
#cash and cash equivalent
cash_and_cash = general_balance_df[general_balance_df['cube.primary-local-name'] == 'CashAndCashEquivalentsModifiedAccrual'].iloc[0]['fact.value']

In [59]:
#receivables
receivables = general_balance_df[general_balance_df['cube.primary-local-name'] == 'AccountsReceivableNetOfAllowanceModifiedAccrual'].iloc[0]['fact.value'] + general_balance_df[general_balance_df['cube.primary-local-name'] == 'PropertyTaxesReceivableModifiedAccrual'].iloc[0]['fact.value']

In [60]:
due_liabilities = general_balance_df[general_balance_df['cube.primary-local-name'] == 'DueToOtherFundsModifiedAccrual'].iloc[0]['fact.value']
total_liabilities = general_balance_df[general_balance_df['cube.primary-local-name'] == 'LiabilitiesModifiedAccrual'].iloc[0]['fact.value']

In [61]:
(cash_and_cash + receivables) / (due_liabilities)

2.9338649929725333

In [62]:
(cash_and_cash + receivables) / (total_liabilities) 

1.3377453975592892

993766